In [1]:
from lightgbm import LGBMClassifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Dataset/test.csv')

In [3]:
df1 = pd.read_csv('Dataset/train.csv')

In [4]:
targets = df1[['pet_id','breed_category','pet_category']]

In [5]:
df1 = df1.drop('breed_category', axis = True)

In [6]:
df1= df1.drop('pet_category', axis = True)

In [7]:
test = pd.concat([df1,df])

In [8]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4


In [9]:
test['issue_date'] = pd.to_datetime(test['issue_date'])

In [10]:
test['listing_date'] = pd.to_datetime(test['listing_date'])

In [11]:
test['days'] =  (test['listing_date'] - test['issue_date']).dt.days

In [12]:
test

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,days
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,73
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,1862
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,752
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,755
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,52
...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10,2017-03-10 14:56:00,2.0,Brown,0.82,36.08,13,9,394
8068,ANSL_59041,2015-12-07,2018-02-12 00:00:00,0.0,Tan,0.49,27.54,13,9,798
8069,ANSL_60034,2015-12-08,2017-01-04 17:19:00,0.0,Black,0.98,37.19,0,7,393
8070,ANSL_58066,2016-06-28,2017-07-20 18:19:00,NaN,Black,0.79,23.83,0,2,387


In [13]:
test['condition'].fillna(1.0, inplace=True)

In [14]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [15]:
test = encode_and_bind(test, 'color_type')

In [16]:
test.drop('color_type',axis = 1, inplace = True)

In [17]:
test['height(m)']=test['height(cm)'].apply(lambda x:x*(.01))

In [18]:
train_df=test[:18834]

In [19]:
test_df= test[18834:]

In [20]:
train_df = pd.concat([train_df,targets], axis =1)

In [21]:
train_df.drop('height(cm)', axis = 1, inplace = True)
test_df.drop('height(cm)', axis = 1, inplace = True)

C:\Users\Ashish Bisht\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
train_df.drop('issue_date', axis = 1, inplace = True)
train_df.drop('listing_date', axis = 1, inplace=True)
test_df.drop('issue_date', axis = 1, inplace = True)
test_df.drop('listing_date', axis = 1, inplace=True)

In [23]:
train_df=train_df.drop(df.columns[0], axis=1)

In [24]:
init_train_df = train_df.drop('pet_category', axis = 1)

In [25]:
X1_train = init_train_df.drop('breed_category', axis = 1)
Y1_test = init_train_df['breed_category']

In [26]:
params = {
        'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
        'gamma': [0.5, 1, 1.5, 2,3,4,5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [1,2,3, 4, 5,6,7],
        
        }

In [27]:
lgm = LGBMClassifier(learning_rate=0.01, n_estimators=800, objective = 'multi:softmax',
                    num_class=3, nthread=1)

In [28]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(lgm, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X1_train,Y1_test), verbose=1, random_state=1001 )


In [29]:
random_search.fit(X1_train,Y1_test)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   22.6s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000029170BBFAC8>,
                   estimator=LGBMClassifier(learning_rate=0.01,
                                            n_estimators=800, nthread=1,
                                            num_class=3,
                                            objective='multi:softmax'),
                   n_iter=5, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 3, 4, 5],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7],
                                        'min_child_weight': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=1001, verbose=1)

In [30]:
test1 = test_df.drop('pet_id', axis =1)

In [31]:
y_test = random_search.predict(test1)

In [32]:
sec_train_df = train_df.drop('breed_category', axis = 1)

In [33]:
X2_train = sec_train_df.drop('pet_category', axis = 1)
Y2_test = sec_train_df['pet_category']

In [34]:
params = {
        'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
        'gamma': [0.5, 1, 1.5, 2,3,4,5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [1,2,3, 4, 5,6,7],
        
        }

In [35]:
lgm = LGBMClassifier(learning_rate=0.01, n_estimators=800, objective = 'multi:softmax',
                    num_class=4, nthread=1)

In [36]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search_1 = RandomizedSearchCV(lgm, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X2_train,Y2_test), verbose=3, random_state=1001 )


In [37]:
random_search_1.fit(X2_train,Y2_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   26.1s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000029170CEB348>,
                   estimator=LGBMClassifier(learning_rate=0.01,
                                            n_estimators=800, nthread=1,
                                            num_class=4,
                                            objective='multi:softmax'),
                   n_iter=5, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 3, 4, 5],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7],
                                        'min_child_weight': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=1001, verbose=3)

In [38]:
y_test1 = random_search_1.predict(test1)

In [39]:
results_df = pd.DataFrame(data={'pet_id':test_df['pet_id'], 'breed_category':y_test,'pet_category':y_test1})
results_df.to_csv('submission_LGBM_new.csv', index=False)